# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'QRDLMGASNTEA)OB ANGCAEKS/S(TL L E',
 'calculationPrice': 'close',
 'open': 134,
 'openTime': 1695029700914,
 'openSource': 'loaffiic',
 'close': 133.69,
 'closeTime': 1648640143790,
 'closeSource': 'faiilfoc',
 'high': 134.9,
 'highTime': 1649021249093,
 'highSource': 'dedn1ui t5r em yaeeipcl',
 'low': 131.11,
 'lowTime': 1665464865050,
 'lowSource': 'nmdeelu eepr1iidt5ac y ',
 'latestPrice': 130.33,
 'latestSource': 'Close',
 'latestTime': 'June 15, 2021',
 'latestUpdate': 1672166161737,
 'latestVolume': 63140605,
 'iexRealtimePrice': 130.845,
 'iexRealtimeSize': 101,
 'iexLastUpdated': 1668179932055,
 'delayedPrice': 133.36,
 'delayedPriceTime': 1701688889613,
 'oddLotDelayedPrice': 133.41,
 'oddLotDelayedPriceTime': 1704058005123,
 'extendedPrice': 134.9,
 'extendedChange': 0.06,
 'extendedChangePercent': 0.00047,
 'extendedPriceTime': 1625476907785,
 'previousClose': 135.04,
 'previousVolume': 101616572,
 'change'

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']
price,market_cap

(130.33, 2234851256709)

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to buy


In [7]:
final_dataframe.append(pd.Series(
    [
    symbol,
    price,
    market_cap,
    'N/A'
    ],
    index=my_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to buy
0,AAPL,130.33,2234851256709,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'    
        ],
        index=my_columns),
    ignore_index=True
    )

KeyboardInterrupt: 

In [9]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to buy
0,A,144.74,45999388248,N/A
1,AAL,23.65,14815288541,N/A
2,AAP,207.22,13587597211,N/A
3,AAPL,132.16,2254652917056,N/A
4,ABBV,118.83,206359933051,N/A
5,ABC,119.96,25159358674,N/A
6,ABMD,304.14,14069013674,N/A
7,ABT,113.88,197045917614,N/A
8,ACN,296.01,197817706477,N/A
9,ADBE,560.97,265773268046,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [11]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index=my_columns
            ),
            ignore_index=True
        )
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to buy
0,A,150.38,45080279896,N/A
1,AAL,23.39,15100078955,N/A
2,AAP,206.38,13239543772,N/A
3,AAPL,131.46,2263318020174,N/A
4,ABBV,118.41,211825960793,N/A
...,...,...,...,...
500,YUM,123.44,35560843641,N/A
501,ZBH,165.64,34131831689,N/A
502,ZBRA,524.80,27845653555,N/A
503,ZION,57.15,9058224669,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [16]:
portfolio_size = input('Enter the Value of your portfolio:')

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \nPlease try again:")
    portfolio_size = input('Enter the Value of your portfolio:')
    val = float(portfolio_size)

Enter the Value of your portfolio:1000


In [18]:
position_size = val/len(final_dataframe.index)
print(position_size)

1.9801980198019802


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: